In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

X = pd.read_csv('/kaggle/input/home-data-for-ml-course/train.csv', index_col='Id') 
X_test = pd.read_csv('/kaggle/input/home-data-for-ml-course/test.csv', index_col='Id')
test_data = pd.read_csv('/kaggle/input/home-data-for-ml-course/test.csv')

X.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X.SalePrice
X.drop(['SalePrice'], axis=1, inplace=True)

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

cc = [cn for cn in X_train.columns if X_train[cn].nunique() < 10 and X_train[cn].dtype == 'object']
nc = [cn for cn in X_train.columns if X_train[cn].dtype in ['int64', 'float64']]

my_cols = cc + nc
X_train = X_train[my_cols].copy()
X_valid = X_valid[my_cols].copy()
X_test = X_test[my_cols].copy()
X = X[my_cols].copy()

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

nt = SimpleImputer(strategy = 'median')
ct = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy = 'constant')),
    ('onehot', OneHotEncoder(handle_unknown = 'ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', nt, nc),
        ('cat', ct, cc)
    ])

model = RandomForestRegressor(n_estimators=100, random_state=0)
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)
                     ])
clf.fit(X_train, y_train)

preds = clf.predict(X_valid)

print('MAE: ', mean_absolute_error(y_valid, preds))

In [ ]:
clf.fit(X, y)
preds_test = clf.predict(X_test)

output = pd.DataFrame({'Id': test_data.Id,
                       'SalePrice': preds_test})
output.to_csv('submission.csv', index=False)